In [ ]:
# Import thư viện
import pandas as pd
import numpy as np

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.available
plt.style.use("seaborn-whitegrid")

# 1. Mô tả bài toán 
Home Depot là một công ty bán lẻ cung cấp thiết bị gia đình của Mỹ chuyên bán dụng cụ, sản phẩm và dịch vụ xây dựng. Khách hàng muốn vào Home Depot để tìm mua thiết bị gia đình. Trong cuộc thi này, Home Depot đang yêu cầu giúp họ cải thiện trải nghiệm mua sắm của khách hàng bằng cách phát triển một mô hình có thể dự đoán mức độ liên quan của kết quả tìm kiếm.

# 2. Phân tích dữ liệu, convert, clean dữ liệu 

## 2.1. Load dữ liệu

Các trường dữ liệu:
1. id: id của một cặp search_term và sản phẩm.
2. product_uid: id của sản phẩm
3. product_title : tên của sản phẩm
4. product_description : mô tả sản phẩm
5. search_term: ô tìm kiếm 
6. relevance : mức độ liên quan tìm kiếm và kết quả
7. name: tên thuộc tính
8. value: giá trị thuộc tính

In [ ]:
#Giải nén tập tin
!unzip ../input/home-depot-product-search-relevance/attributes.csv.zip
!unzip ../input/home-depot-product-search-relevance/product_descriptions.csv.zip
!unzip ../input/home-depot-product-search-relevance/test.csv.zip
!unzip ../input/home-depot-product-search-relevance/train.csv.zip
!unzip ../input/home-depot-product-search-relevance/sample_submission.csv.zip

### 2.1.1 Load từ file train
Là dữ liệu của tập train, bao gồm sản phẩm, ô tìm kiếm và điểm relevance

In [ ]:
train_df = pd.read_csv("./train.csv", encoding="ISO-8859-1")
train_df.head()

#### Tập dữ liệu train gồm 74067 giá trị và không có giá trị nào rỗng

In [ ]:
train_df.info()

### 2.1.2 Load từ file test
Là dữ liệu của file test, bao gồm sản phẩm, ô tìm kiếm và ở nhiệm vụ của ta là tính toán điểm relevance cho file này.

In [ ]:
test_df = pd.read_csv("./test.csv", encoding="ISO-8859-1")
test_df.head()

#### Tập dữ liệu train gồm 166693 giá trị và không có giá trị nào rỗng

In [ ]:
test_df.info()

### 2.1.3 Load từ file submission
Là mẫu file submit.

In [ ]:
submission = pd.read_csv("./sample_submission.csv", index_col=["id"])
submission.head()

### 2.1.4 Load từ file attributes
Là file dữ liệu gồm các mô tả chi tiết các thuộc tính của từng sản phẩm

In [ ]:
attributes_df = pd.read_csv("./attributes.csv", encoding="ISO-8859-1")
attributes_df.head()

In [ ]:
attributes_df.name.value_counts().head(10)

#### Ta thấy các hàng như Brand Name hay Bullet đều xuất hiện rất nhiều lần. Đây là những thông tin rất quan trọng

#### Tập dữ liệu train gồm 2044803 giá trị

In [ ]:
attributes_df.info()

### 2.1.5 Load từ file descriptions
Là file dữ liệu gồm các mô tả của từng sản phẩm

In [ ]:
product_df = pd.read_csv("./product_descriptions.csv", encoding="ISO-8859-1")
product_df.head()

#### Tập dữ liệu train gồm 124428 giá trị và không có giá trị nào rỗng

In [ ]:
product_df.info()

In [ ]:
! rm ./train.csv
! rm ./product_descriptions.csv
! rm ./sample_submission.csv
! rm ./test.csv

## 2.2 Miêu tả bộ dữ liệu

In [ ]:
# Phân bố của relevance
sns.countplot(x="relevance", data = train_df)
plt.show()

#### Điểm relevance nằm chủ yếu ở khoảng giá trị [2.0, 3.0]

In [ ]:
# Số từ mỗi câu tìm kiếm
word_length = [len(x.split()) for x in train_df["search_term"]]
fig, ax = plt.subplots()
ax.hist(word_length)
ax.set(title="word count", xlabel="Number of word")
plt.show()

#### Độ dài mỗi câu tìm kiếm phần lớn từ 2 đến 4 từ

## 3. Làm sạch dữ liệu

1. Nối dữ liệu hai bảng attributes_df và product_df vào bảng train_df 
2. Chuẩn hóa các từ 
3. Loại bỏ các từ stopword và stemming

### 3.1 Nối dữ liệu hai bảng attributes_df và product_df vào bảng train_df

In [ ]:
# Nối dữ liệu bảng product_df
train_df = train_df.merge(product_df, on = "product_uid")

In [ ]:
train_df.head()

In [ ]:
# Nối dữ liệu bảng attributes_df
# Tạo một bảng mới tên là df giống với bảng attribute
df = attributes_df
# Tạo cột dữ liệu mới là attribute là tổng hai cột name và value
df['attribute'] = df['name'] + " " + df['value']
# Cộng tất cả các attribute của từng sản phẩm vào làm một
df_att = df.groupby('product_uid').agg({'attribute': lambda s : ' '.join(s.astype(str))}).reset_index()
df_att.info()

In [ ]:
df_att.head()

In [ ]:
# Nối dữ liệu bảng df_att vào bảng train_df
train_df = train_df.merge(df_att, on="product_uid", how="left")
train_df.info()

#### Ta thấy cột attribute đang có giá trị rỗng
Với những giá trị rỗng đó ta điền thành ""

In [ ]:
train_df["attribute"] = train_df["attribute"].fillna("")

In [ ]:
train_df.info()

In [ ]:
train_df.head()

### 3.3 Chuẩn hóa các từ
1. Chuẩn hóa các đơn vị đo ("in.", "in",.. => "inch"), các số
2. Xóa các ký tự đặc biệt 

In [ ]:
import re

#stopwords are the words contain very little or no imformation. 
stop_w = ['for', 'xbi', 'and', 'in', 'th','on','sku','with','what','from','that','less','er','ing'] #'electr','paint','pipe','light','kitchen','wood','outdoor','door','bathroom'

strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}
CLEANR = re.compile('<.*?>') 

def remove_html_tag(text):
  cleantext = re.sub(CLEANR, '', text)
  return cleantext

def str_stem(s):
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
# Chuyển 2 khoảng trắng "  " thành 1 khoảng trắng " "        
        s = s.replace("  "," ") 
        
#Xóa các ký tự đặc biệt
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        
# Chuẩn hóa các đơn vị
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        
# Chuyển các sô bằng chữ thành số
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
# Xóa các tag html
        s = remove_html_tag(s)
# Chuyển các chữ hoa thành chữ thường
        s = s.lower()
        return s
    else:
        return "null"

In [ ]:
# Chuẩn hóa trên các cột của train_df
train_df["product_title"] = train_df["product_title"].apply(lambda x : str_stem(x))
train_df["search_term"] = train_df["search_term"].apply(lambda x : str_stem(x))
train_df["attribute"] = train_df["attribute"].apply(lambda x : str_stem(x))
train_df["product_description"] = train_df["product_description"].apply(lambda x : str_stem(x))

In [ ]:
# File train_df sau khi chuẩn hóa
train_df.head()

### 3.3 Loại bỏ stop word và stemming
Stop word là những từ xuất hiện nhiều mà không mang hoặc mang rất ít thông tin như "me", "she", "the",... Loại bỏ các stop word sẽ giúp dữ liệu sạch và nhẹ hơn

Stemming là kỹ thuật dùng để biến đổi 1 từ về dạng gốc bằng cách loại bỏ 1 số ký tự nằm ở cuối từ mà nó nghĩ rằng là biến thể của từ. "walked", "walking", "walks" chỉ khác nhau là ở những ký tự cuối cùng, bằng cách bỏ đi các hậu tố –ed, –ing hoặc –s, chúng ta sẽ được từ nguyên gốc là "walk".

Ở đây em sử dụng thư viện nltk để loại bỏ stop word và stemming

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Sử dụng Lancaster Stemmer
from nltk.stem import LancasterStemmer

stop_words = set(stopwords.words('english'))
lancaster = LancasterStemmer()

def rm_stopword(text):
    word_tokens = word_tokenize(text)
    word_tokens = list(dict.fromkeys(word_tokens))
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    rt = ""
    for s in filtered_sentence:
        s = lancaster.stem(s)
        rt += s + " "
    return rt

In [ ]:
# Loại bỏ stop word và stemming trong train_df
train_df["product_title"] = train_df["product_title"].apply(lambda x : rm_stopword(x))
train_df["search_term"] = train_df["search_term"].apply(lambda x : rm_stopword(x))
train_df["attribute"] = train_df["attribute"].apply(lambda x : rm_stopword(x))
train_df["product_description"] = train_df["product_description"].apply(lambda x : rm_stopword(x))

In [ ]:
# Tập train_df sau khi xử lý
train_df.head()

# 4. Trích xuất các đặc trưng
1. Len of query
2. overlap title
3. overlap des
4. overlap att
5. levenshtein
6. average word2vec

## 4.1 Độ dài của các trường "search_term", "product_title", "product_description"

In [ ]:
train_df.shape[0]

In [ ]:
leng_search_term = []
leng_product_title = []
leng_product_description = []

# train_df.shape[0] = 74067
for i in range(0, train_df.shape[0]):
#     Lấy độ dài từng ô rồi nối vào mảng
    leng_search_term.append(len(train_df.loc[i][3].split()))
    leng_product_title.append(len(train_df.loc[i][2].split()))
    leng_product_description.append(len(train_df.loc[i][5].split()))

In [ ]:
# Nối vào bảng train_df
train_df["search_count"] = leng_search_term
train_df["title_count"] = leng_product_title
train_df["des_count"] = leng_product_description

In [ ]:
train_df.head()

## 4.2 . Tỉ lệ khớp

Tỉ lệ khớp (overlap) là một kỹ thuật tính phần trăm của từ xuất hiện ở ô tìm kiếm có trong tiêu đề, mô tả, ... 

Được tính bằng công thức: overlap = (từ trong ô tìm kiếm xuất hiện trong các ô tiêu đề,...)/(số từ có trong ô tìm kiếm)

In [ ]:
overlap_title = []
overlap_des = []
overlap_att = []

# train_df.shape[0] = 74067
for i in range(0, train_df.shape[0]):
    search_length = train_df.loc[i][7]
    word_tokens = word_tokenize(train_df.loc[i][3])
    tit = 0
    des = 0
    att = 0
    for s in word_tokens:
#         title
# Nếu xuất hiện thì là 1, còn lại thì không 
        check = 0 if train_df.loc[i][2].count(s) == 0 else 1
        tit += check/search_length
#         des
        check = 0 if train_df.loc[i][5].count(s) == 0 else 1
        des += check/search_length
#         attribute
        check = 0 if train_df.loc[i][6].count(s) == 0 else 1
        att += check/search_length
        
    overlap_att.append(att)
    overlap_des.append(des)
    overlap_title.append(tit)
      

In [ ]:
# Nối vào bảng train_df
train_df["overlap_title"] = overlap_title
train_df["overlap_des"] = overlap_des
train_df["overlap_att"] = overlap_att

## 4.3  Levenshtein

Tính khoảng cách chỉnh sửa Levenshtein giữa hai chuỗi. Khoảng cách chỉnh sửa là số ký tự cần được thay thế, chèn hoặc xóa để biến đổi s1 thành s2. Ví dụ: chuyển đổi "rain" thành "shine" yêu cầu ba bước, bao gồm hai thay thế và một lần chèn: “rain” -> “sain” -> “shin” -> “shine”

Ở đây em sử dụng hàm nltk.edit_distance() của thư viện nltk

Tính Levenshtein giữa ô tìm kiếm với product_title và product_description

In [ ]:
#  levenshtein
import nltk
lev_title = []
lev_des = []


for i in range(0, train_df.shape[0]):
#     Levenshtein với product_title
    lev_title.append(nltk.edit_distance(train_df.loc[i][3], train_df.loc[i][2]))
#  Levenshtein với product_description
    lev_des.append(nltk.edit_distance(train_df.loc[i][3], train_df.loc[i][5]))

In [ ]:
# Nối với train_df
train_df["lev_title"] = lev_title
train_df["lev_des"] = lev_des

In [ ]:
# Tập train sau khi được xử lý
train_df.head()

# 5. Mô tả thuật toán, mô hình được chọn 

## 5.1 Tách dữ liệu train

In [ ]:
# Tên các feature
feature = ["product_uid", "search_count", "title_count" , "des_count", "overlap_title", "overlap_des", "overlap_att", "lev_title", "lev_des"]

In [ ]:
# Tập dữ liệu x train
x_train = train_df[feature]
# Tập dữ liệu y train
y_train = train_df["relevance"]

In [ ]:
y_train.head()

In [ ]:
x_train.head()

## 5.2 Xử lý tập dữ liệu test

Ta xử lý tập test và trích xuất đặc trưng như đối với tập train

In [ ]:
test_df.info()

In [ ]:
# Nối bảng
test_df = test_df.merge(product_df, on = "product_uid")
test_df = test_df.merge(df_att, on="product_uid", how="left")
test_df["attribute"] = test_df["attribute"].fillna("")

# Tiền xử lý dữ liệu
test_df["product_title"] = test_df["product_title"].apply(lambda x : str_stem(x))
test_df["search_term"] = test_df["search_term"].apply(lambda x : str_stem(x))
test_df["attribute"] = test_df["attribute"].apply(lambda x : str_stem(x))
test_df["product_description"] = test_df["product_description"].apply(lambda x : str_stem(x))

leng_search_term = []
leng_product_title = []
leng_product_description = []

# train_df.shape[0] = 74067
for i in range(0, test_df.shape[0]):
#     Lấy độ dài từng ô rồi nối vào mảng
    leng_search_term.append(len(test_df.loc[i][3].split()))
    leng_product_title.append(len(test_df.loc[i][2].split()))
    leng_product_description.append(len(test_df.loc[i][4].split()))
    
# Nối vào bảng train_df
test_df["search_count"] = leng_search_term
test_df["title_count"] = leng_product_title
test_df["des_count"] = leng_product_description

In [ ]:
overlap_title = []
overlap_des = []
overlap_att = []

for i in range(0, test_df.shape[0]):
    search_length = test_df.loc[i][6]
    word_tokens = word_tokenize(test_df.loc[i][3])
    tit = 0
    des = 0
    att = 0
    for s in word_tokens:
#         title
        check = 0 if test_df.loc[i][2].count(s) == 0 else 1
        tit += check/search_length
#         des
        check = 0 if test_df.loc[i][4].count(s) == 0 else 1
        des += check/search_length
#         attribute
        check = 0 if test_df.loc[i][5].count(s) == 0 else 1
        att += check/search_length
        
    overlap_att.append(att)
    overlap_des.append(des)
    overlap_title.append(tit)

# Nối vào bảng train_df
test_df["overlap_title"] = overlap_title
test_df["overlap_des"] = overlap_des
test_df["overlap_att"] = overlap_att    

test_df.head()

In [ ]:
lev_title = []
lev_des = []

for i in range(0, test_df.shape[0]):
#   Levenshtein với product_title
    lev_title.append(nltk.edit_distance(test_df.loc[i][3], test_df.loc[i][2]))
#  Levenshtein với product_description
    lev_des.append(nltk.edit_distance(test_df.loc[i][3], test_df.loc[i][4]))

# Nối vào test
test_df["lev_title"] = lev_title
test_df["lev_des"] = lev_des

In [ ]:
# Tập test sau khi xử lý
test_df.head()

### Tách dữ liệu tập test

In [ ]:
# Gồm các feature giống tập train
x_test = test_df[feature]

## 5.3 Mô hình

Em sử dụng mô hình Random Forests trong thư viện Scikit-learn. Random Forests là thuật toán học có giám sát. Thuật toán sẽ gồm nhiều cây quyết định. Sau đó kết quả sẽ được tổng hợp từ cây quyết định. Random forests tạo ra cây quyết định trên các mẫu dữ liệu được chọn ngẫu nhiên, được dự đoán từ mỗi cây và chọn giải pháp tốt nhất bằng cách bỏ phiếu. 

Các tham số của mô hình:
1. max_depth: độ sâu của cây quyết định
2. max_features: số lượng các features cần xem xét để phân chia tốt nhất
3. n_estimators: số lượng cây trong mô hình
4. n_jobs: Số lượng công việc phải chạy song song. -1 là sử dụng tất cả
5. learning_rate: hệ số học, hệ số này dùng để nhân với kết quả mô hình trước
6. random_state: Kiểm soát cả tính ngẫu nhiên của khởi động chuỗi các mẫu được sử dụng khi xây dựng cây

In [ ]:
# Hàm chuẩn hóa những giá trị relavane dự đoán ngoài khoảng cho phép
def nor(x):
    if x < 1:
        return 1
    elif x > 3:
        return 3
    return x

In [ ]:
# randomforest
# import thư viện 
from sklearn.ensemble import RandomForestRegressor
# Khởi tạo model
model=RandomForestRegressor(n_jobs= -1,n_estimators= 1000,random_state=37,max_depth = 9.0, max_features = 0.997677)
# Fit dữ liệu vào model
model.fit(x_train,y_train)
# Dự đoán model với dữ liệu x_test
y_predict = model.predict(x_test)

# 6. Báo cáo

## 6.1 Mức độ quan trọng của các features

In [ ]:
feature_imp = pd.Series(model.feature_importances_,index=feature).sort_values(ascending=False)
feature_imp

In [ ]:
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
# plt.legend()
plt.show()

## 6.2 Các mô hình khác

Em thử mô hình XGBRegressor và GradientBoostingRegressor với tham số:
1.XGBRegressor(colsample_bytree=0.4,learning_rate=0.1,max_depth=6,n_estimators=700,reg_alpha=0.075,reg_lambda=0.045,subsample=0.6,seed=42)
2.GradientBoostingRegressor(n_estimators=700,max_depth=6, random_state=42)

## 6.3 Xuất file kết quả

In [ ]:
y_predict

In [ ]:
# Gán giá trị vừa dự đoán vào bảng submit
submission["relevance"] = y_predict
# Chuẩn hóa relevance
submission["relevance"] = submission["relevance"].apply(lambda x: nor(x))
# Ghi ra file để submit
submission.to_csv("submission.csv")